# Divergent data reduction for Amor

In this notebook, we will look at how to use the `essreflectometry` package with Sciline, for reflectometry data collected from the PSI instrument [Amor](https://www.psi.ch/en/sinq/amor) in [divergent beam mode](https://www.psi.ch/en/sinq/amor/selene).

We will begin by importing the modules that are necessary for this notebook.

## Setup

In [ ]:
import warnings
import scipp as sc
from ess import amor
from ess.amor import data  # noqa: F401
from ess.reflectometry.types import *
from ess.amor.types import *

# The files used in this tutorial have some issues that makes scippnexus
# raise warnings when loading them. To avoid noise in the notebook the warnings are silenced.
warnings.filterwarnings('ignore', 'Failed to convert .* into a transformation')
warnings.filterwarnings('ignore', 'Invalid transformation, missing attribute')

## Create and configure the workflow

We begin by creating the Amor workflow object which is a skeleton for reducing Amor data,
with pre-configured steps, and then set the missing parameters which are specific to each experiment:

In [ ]:
workflow = amor.AmorWorkflow()
workflow[SampleSize[SampleRun]] = sc.scalar(10.0, unit='mm')
workflow[SampleSize[ReferenceRun]] = sc.scalar(10.0, unit='mm')

workflow[ChopperPhase[ReferenceRun]] = sc.scalar(-7.5, unit='deg')
workflow[ChopperPhase[SampleRun]] = sc.scalar(-7.5, unit='deg')

workflow[QBins] = sc.geomspace(dim='Q', start=0.005, stop=0.3, num=391, unit='1/angstrom')
workflow[WavelengthBins] = sc.geomspace('wavelength', 2.8, 12.5, 2001, unit='angstrom')

# The YIndexLimits and ZIndexLimits define ranges on the detector where
# data is considered to be valid signal.
# They represent the lower and upper boundaries of a range of pixel indices.
workflow[YIndexLimits] = sc.scalar(11), sc.scalar(41)
workflow[ZIndexLimits] = sc.scalar(80), sc.scalar(370)
workflow[BeamDivergenceLimits] = sc.scalar(-0.75, unit='deg'), sc.scalar(0.75, unit='deg')

In [ ]:
workflow.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

## Caching the reference result

The reference result (used for normalizing the sample data) only needs to be computed once.
It represents the intensity reflected by the super-mirror.

We compute it using the pipeline and thereafter set the result back on the original pipeline.

In [ ]:
workflow[Filename[ReferenceRun]] = amor.data.amor_reference_run()
# The sample rotation value in the file is slightly off, so we set it manually
workflow[SampleRotation[ReferenceRun]] = sc.scalar(0.65, unit='deg')

reference_result = workflow.compute(ReducedReference)
# Set the result back onto the pipeline to cache it
workflow[ReducedReference] = reference_result

If we now visualize the pipeline again, we can see that the reference is not re-computed:

In [ ]:
workflow.visualize(ReflectivityOverQ, graph_attr={'rankdir': 'LR'})

## Computing sample reflectivity

We now compute the sample reflectivity from 4 runs that used different sample rotation angles.
The measurements at different rotation angles cover different ranges of $Q$.

In this tutorial we use some Amor data files we have received.
The file paths to the tutorial files are obtained by calling:

In [ ]:
amor.data.amor_sample_run('608')

When you encounter `amor.data.amor_sample_run` you should imagining replacing that with a path to your own dataset.

In [ ]:
runs = {
    '608': {
        # The sample rotation values in the files are slightly off, so we replace
        # them with corrected values.
        SampleRotation[SampleRun]: sc.scalar(0.85, unit='deg'),
        Filename[SampleRun]: amor.data.amor_sample_run('608'),
    },
    '609': {
        SampleRotation[SampleRun]: sc.scalar(2.25, unit='deg'),
        Filename[SampleRun]: amor.data.amor_sample_run('609'),
    },
    '610': {
        SampleRotation[SampleRun]: sc.scalar(3.65, unit='deg'),
        Filename[SampleRun]: amor.data.amor_sample_run('610'),
    },
    '611': {
        SampleRotation[SampleRun]: sc.scalar(5.05, unit='deg'),
        Filename[SampleRun]: amor.data.amor_sample_run('611'),
    },
}


reflectivity = {}
for run_number, params in runs.items():
    workflow[Filename[SampleRun]] = params[Filename[SampleRun]]
    workflow[SampleRotation[SampleRun]] = params[SampleRotation[SampleRun]]
    reflectivity[run_number] = workflow.compute(ReflectivityOverQ).hist()

sc.plot(reflectivity, norm='log', vmin=1e-4)

## Scaling the reflectivity curves to overlap

In case we know the curves are have been scaled by different factors (that are constant in Q) it can be useful to scale them so they overlap:

In [ ]:
from ess.reflectometry.tools import scale_reflectivity_curves_to_overlap

scaled_reflectivity_curves, scale_factors = scale_reflectivity_curves_to_overlap(
    reflectivity.values(),
    # Optionally specify a Q-interval where the reflectivity is known to be 1.0
    critical_edge_interval=(sc.scalar(0.01, unit='1/angstrom'), sc.scalar(0.014, unit='1/angstrom'))
)

sc.plot(dict(zip(reflectivity.keys(), scaled_reflectivity_curves, strict=True)), norm='log', vmin=1e-5)

Curves obtained from measurements at different angles can be combined to one common reflectivity curve:

In [ ]:
from ess.reflectometry.tools import combine_curves
combined = combine_curves(scaled_reflectivity_curves, workflow.compute(QBins))
combined.plot(norm='log')

## Diagnostic figures

There are some useful visualizations that can be used to troubleshoot the instrument.
They typically operate on the `ReflectivityData`.

The difference between `ReflectivityData` and `ReflectivityOverQ` is that `ReflectivityData` is not binned in $Q$, but instead has the same shape as the reference.

Essentially it represents a "reflectivity" computed in every wavelength-detector coordinate (`z_index`) bin.
This makes it easier to spot inhomogeneities and diagnose problems.

In [ ]:
# Start by computing the `ReflectivityData` for each of the files
diagnostics = {}
for run_number, params in runs.items():
    workflow[Filename[SampleRun]] = params[Filename[SampleRun]]
    workflow[SampleRotation[SampleRun]] = params[SampleRotation[SampleRun]]
    diagnostics[run_number] = workflow.compute((ReflectivityOverZW, ThetaBins[SampleRun]))

# Scale the results using the scale factors computed earlier
for run_number, scale_factor in zip(reflectivity.keys(), scale_factors, strict=True):
    diagnostics[run_number][ReflectivityOverZW] *= scale_factor

In [ ]:
diagnostics['608'][ReflectivityOverZW].hist().flatten(('blade', 'wire'), to='z').plot(norm='log')

### Make a $(\lambda, \theta)$ map
A good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $\theta$ bins and make sure the triangles converge at the origin.

In [ ]:
from ess.amor.figures import wavelength_theta_figure

wavelength_theta_figure(
    [result[ReflectivityOverZW] for result in diagnostics.values()],
    theta_bins=[result[ThetaBins[SampleRun]] for result in diagnostics.values()],
    q_edges_to_display=(sc.scalar(0.018, unit='1/angstrom'), sc.scalar(0.113, unit='1/angstrom'))
)

This plot can be used to check if the value of the sample rotation angle $\omega$ is correct. The bright triangles should be pointing back to the origin $\lambda = \theta = 0$. In the figure above the black lines are all passing through the origin.

### Make a $(Q, \theta)$ map
Another good sanity check is to create a two-dimensional map of the counts in $\lambda$ and $Q$ and make sure the stripes are vertical. If they are not that could indicate that the `ChopperPhase` setting is incorrect.

In [ ]:
from ess.amor.figures import q_theta_figure

q_theta_figure(
    [res[ReflectivityOverZW] for res in diagnostics.values()],
    theta_bins=[res[ThetaBins[SampleRun]] for res in diagnostics.values()],
    q_bins=workflow.compute(QBins)
)

### Compare the sample measurement to the reference on the detector

Here we compare the raw number of counts on the detector for the sample measurement and the reference respectively.

`z_index` is the $z-$detector coordinate, that is, the detector coordinate in the direction of the scattering angle $\theta$.

In [ ]:
from ess.amor.figures import wavelength_z_figure

workflow[Filename[SampleRun]] = runs['608'][Filename[SampleRun]]
workflow[SampleRotation[SampleRun]] = runs['608'][SampleRotation[SampleRun]]
wavelength_z_figure(
    workflow.compute(Sample),
    wavelength_bins=workflow.compute(WavelengthBins),
    grid=False
) + wavelength_z_figure(
     workflow.compute(Reference),
    grid=False
)

## Save data

We can save the computed $I(Q)$ to an [ORSO](https://www.reflectometry.org) [.ort](https://github.com/reflectivity/file_format/blob/master/specification.md) file using the [orsopy](https://orsopy.readthedocs.io/en/latest/index.html) package.

First, we need to collect the metadata for that file.
To this end, we build a pipeline with additional providers.
We also insert a parameter to indicate the creator of the processed data.

In [ ]:
from ess.reflectometry import orso
from orsopy import fileio

In [ ]:
workflow[orso.OrsoCreator] = orso.OrsoCreator(
    fileio.base.Person(
        name='Max Mustermann',
        affiliation='European Spallation Source ERIC',
        contact='max.mustermann@ess.eu',
    )
)

In [ ]:
workflow.visualize(orso.OrsoIofQDataset, graph_attr={'rankdir': 'LR'})

We build our ORSO dataset from the computed $I(Q)$ and the ORSO metadata:

In [ ]:
iofq_dataset = workflow.compute(orso.OrsoIofQDataset)
iofq_dataset

We also add the URL of this notebook to make it easier to reproduce the data:

In [ ]:
iofq_dataset.info.reduction.script = (
    'https://scipp.github.io/essreflectometry/examples/amor.html'
)

Now let's repeat this for all the sample measurements!
To do that we can use an utility in `ess.reflectometry.tools`:

In [ ]:
from ess.reflectometry.tools import orso_datasets_from_measurements

datasets = orso_datasets_from_measurements(
    workflow,
    runs.values(),
    # Optionally scale the curves to overlap using `scale_reflectivity_curves_to_overlap`
    scale_to_overlap=True
)

Finally, we can save the data to a file.
Note that `iofq_dataset` is an [orsopy.fileio.orso.OrsoDataset](https://orsopy.readthedocs.io/en/latest/orsopy.fileio.orso.html#orsopy.fileio.orso.OrsoDataset).

In [ ]:
fileio.orso.save_orso(datasets=datasets, fname='amor_reduced_iofq.ort')

Look at the first 50 lines of the file to inspect the metadata:

In [ ]:
!head amor_reduced_iofq.ort -n50